<a href="https://colab.research.google.com/github/t84210016-oss/deetect/blob/main/detectoffers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from IPython.utils.text import textwrap
import json


In [34]:

class Classifier:
    def __init__(self, model_type="paraphrase-multilingual-MiniLM-L12-v2", offer_keys=[], seeker_keys=[]):
        self.model = SentenceTransformer(model_type)
        # encode offers and seekers
        self.offer_vecs = self.model.encode(offer_keys)
        self.seeker_vecs = self.model.encode(seeker_keys)

    def classify_post(self, post):
        vec = self.model.encode(post)
        offer_score = util.cos_sim(vec, self.offer_vecs).max()
        seeker_score = util.cos_sim(vec, self.seeker_vecs).max()

        if offer_score > seeker_score:
            return "OFFER", float(offer_score)
        else:
            return "SEEKER", float(seeker_score)


In [35]:


class Imp:
    def __init__(self, json_path, json_offers, json_seekers):

        with open(json_path, "r", encoding="utf-8") as f:
            self.posts = json.load(f)

        with open(json_offers, "r", encoding="utf-8") as f:
            self.offer_keys = json.load(f)

        with open(json_seekers, "r", encoding="utf-8") as f:
            self.seeker_keys = json.load(f)

        self.classifier = Classifier(
            model_type="paraphrase-multilingual-MiniLM-L12-v2",
            offer_keys=self.offer_keys,
            seeker_keys=self.seeker_keys
        )
        self.data_for_df = []

    def to_df(self):
        self.data_for_df = []  # reset in case of multiple calls
        for email, content in self.posts.items():
            text = content["post_text"]
            label, score = self.classifier.classify_post(text)
            self.data_for_df.append({
                "email": email,
                "keyword": content.get("keyword", ""),
                "post_text": text,
                "classification": label,
                "score": score
            })
        return pd.DataFrame(self.data_for_df)


    def output(self):
      d=pd.DataFrame(self.data_for_df)
      d_offers=d[d["classification"]=="OFFER"]
      df_offers=d_offers[['email','post_text']]
      df_offers.to_json("offers.json", orient="records", lines=True, force_ascii=False)




In [36]:
imp =Imp("/content/linkedin_emails00000.json","/content/offers.json" ,"/content/seekers.json")
df=imp.to_df()
imp.output()

JSONDecodeError: Extra data: line 2 column 1 (char 724)

In [ ]:
df.head()